<a href="https://colab.research.google.com/github/yasirabd/solver-society-job-data/blob/main/2_1_Feature_selection_%26_data_cleaning_pada_Jobstreet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objective
Feature selection dan cleaning data.

Data input yang dibutuhkan:
1. jobstreet_4_oct.csv
2. waktu_bekerja_manual_perbaikan.csv
3. gaya_berpakaian_mapper.csv
4. tunjangan_item_mapper2.csv
5. tunjangan_item_mapper3.csv
6. bahasa_mapper.csv
7. daftar_kota_kabupaten_id.csv


Data output yang dihasilkan:
1. jobstreet_clean_tahap1.csv

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [ ]:
jobstreet = pd.read_csv("/content/drive/My Drive/Data Loker/jobstreet_4oct.csv")
jobstreet.head()

,platform,job_position,company_name,years_of_experience,company_location,company_address,posting_date,closing_date,job_description,average_processing_time,company_industry,company_site,company_size,work_environment_waktu_bekerja,work_environment_gaya_berpakaian,work_environment_tunjangan,work_environment_bahasa,company_overview
0,jobstreet,SPV FINANCE,PT Axindo Infotama,Min 3 tahun (Supervisor/Koordinator),Indonesia - Jakarta Raya - Jakarta Pusat - Ruk...,Ruko Mega Grosir Cempaka Mas Blok K No. 29-30,04-October-2020,Tutup pada 03-November-2020,[SEP][SEP]Usia Max. 35 tahun[SEP]Pendidikan Mi...,6 hari,Retail/Merchandise,http://axindo.co.id,51 - 200 pekerja,Sabtu atau memerlukan shift,Formil (contoh: Kemeja + Dasi),"Kesehatan, Parkir",Bahasa Indonesia,[SEP][SEP] PT. Axindo Infotama didirikan di Ja...
1,jobstreet,Qualitative Research Executive,Perusahaan Dirahasiakan,Min 3 tahun (Pegawai (non-manajemen & non-supe...,Indonesia - Jakarta Raya,NaN,04-October-2020,Tutup pada 03-November-2020,RESPONSIBILITIES:[SEP]Brief Description of Dut...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[SEP]We are an international market research c...
2,jobstreet,Field Collection (JS - FC) Wonogiri,OTO Group,Lulusan baru/Pengalaman kerja kurang dari 1 tahun,Indonesia - Jawa Tengah - Wonogiri Wonogoro - ...,"Gedung Summit Mas II lantai 18, Jl. Jend Soedi...",24-September-2020,Tutup pada 24-October-2020,[SEP]Lingkup pekerjaan: [SEP]Menjaga account d...,Lebih dari 2 minggu,Perbankan/Pelayanan Keuangan,http://www.karir.oto.co.id,Lebih dari 5000 pekerja,"Waktu regular, Senin - Jumat",Bisnis (contoh: Kemeja),"Kesehatan, Gigi, bonus tahunan",Bahasa Indonesia,[SEP]OTO Group is a subsidiary company of SUMI...
3,jobstreet,Perawat di Negara Jerman,PT Jerman Manajemen,Min 1 tahun (Pegawai (non-manajemen & non-supe...,Indonesia - Jakarta Raya,"Komplek Balikpapan Baru, Ruko Sentra Eropa Blo...",04-October-2020,Tutup pada 03-November-2020,[SEP]Candidate must possess at least Diploma i...,11 hari,Pendidikan,http://www.jermanmanajemen.de,51 - 200 pekerja,SABTU-KAMIS,Formil (contoh: Kemeja + Dasi),Kesehatan,Bahasa Indonesia,[SEP]Kami menghimbau agar Anda berhati-hati sa...
4,jobstreet,Field Collection (JS - FC) Pati,OTO Group,Lulusan baru/Pengalaman kerja kurang dari 1 tahun,Indonesia - Jawa Tengah - Pati,"Gedung Summit Mas II lantai 18, Jl. Jend Soedi...",24-September-2020,Tutup pada 24-October-2020,[SEP]Lingkup pekerjaan: [SEP]Menjaga account d...,Lebih dari 2 minggu,Perbankan/Pelayanan Keuangan,http://www.karir.oto.co.id,Lebih dari 5000 pekerja,"Waktu regular, Senin - Jumat",Bisnis (contoh: Kemeja),"Kesehatan, Gigi, bonus tahunan",Bahasa Indonesia,[SEP]OTO Group is a subsidiary company of SUMI...


# Feature selection

| No | Attributes                       | Keterangan                                                  |
|----|----------------------------------|-------------------------------------------------------------|
| 1  | platform                         | Tempat data diambil (jobstreet)                             |
| 2  | job_position                     | Posisi pekerjaan yang dicari perusahaan                     |
| 3  | company_name                     | Nama perusahaan yang membuka lowongan kerja                 |
| 4  | years_of_experience              | Lama pengalaman yang dibutuhkan untuk apply pekerjaan       |
| 5  | company_location                 | Lokasi perusahaan yang membuka lowongan kerja               |
| 6  | company_address                  | Alamat perusahaan                                           |
| 7  | posting_date                     | Waktu perusahaan posting lowongan kerja pada platform       |
| 8  | closing_date                     | Waktu post lowongan kerja selesai ditampilkan pada platform |
| 9  | job_description                  | Deskripsi dari kualifikasi dan tanggung jawab pekerjaan     |
| 10 | average_processing_time          | Lama waktu perusahaan memproses lamaran kerja               |
| 11 | company_industry                 | Jenis industri perusahaan                                   |
| 12 | company_site                     | Website perusahaan                                          |
| 13 | company_size                     | Ukuran perusahaan dari jumlah karyawan                      |
| 14 | work_environment_waktu_bekerja   | Waktu bekerja karyawan                                      |
| 15 | work_environment_gaya_berpakaian | Pakaian yang digunakan karyawan formal/non-formal           |
| 16 | work_environment_tunjangan       | Tunjangan yang diberikan pada karyawan                      |
| 17 | work_environment_bahasa          | Bahasa yang digunakan pada perusahaan                       |
| 18 | company_overview                 | Deskripsi perusahaan                                        |

Berdasarkan diskusi 5 Oktober 2020, atribut yang **tidak dibutuhkan** adalah:
1. company_address
2. posting_date
3. closing_date
4. company_site
5. company_overview

In [ ]:
att_removed = ['company_address', 'posting_date', 'closing_date', 
               'company_site', 'company_overview']

data = jobstreet.drop(att_removed, axis=1)
data.head()

,platform,job_position,company_name,years_of_experience,company_location,job_description,average_processing_time,company_industry,company_size,work_environment_waktu_bekerja,work_environment_gaya_berpakaian,work_environment_tunjangan,work_environment_bahasa
0,jobstreet,SPV FINANCE,PT Axindo Infotama,Min 3 tahun (Supervisor/Koordinator),Indonesia - Jakarta Raya - Jakarta Pusat - Ruk...,[SEP][SEP]Usia Max. 35 tahun[SEP]Pendidikan Mi...,6 hari,Retail/Merchandise,51 - 200 pekerja,Sabtu atau memerlukan shift,Formil (contoh: Kemeja + Dasi),"Kesehatan, Parkir",Bahasa Indonesia
1,jobstreet,Qualitative Research Executive,Perusahaan Dirahasiakan,Min 3 tahun (Pegawai (non-manajemen & non-supe...,Indonesia - Jakarta Raya,RESPONSIBILITIES:[SEP]Brief Description of Dut...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,jobstreet,Field Collection (JS - FC) Wonogiri,OTO Group,Lulusan baru/Pengalaman kerja kurang dari 1 tahun,Indonesia - Jawa Tengah - Wonogiri Wonogoro - ...,[SEP]Lingkup pekerjaan: [SEP]Menjaga account d...,Lebih dari 2 minggu,Perbankan/Pelayanan Keuangan,Lebih dari 5000 pekerja,"Waktu regular, Senin - Jumat",Bisnis (contoh: Kemeja),"Kesehatan, Gigi, bonus tahunan",Bahasa Indonesia
3,jobstreet,Perawat di Negara Jerman,PT Jerman Manajemen,Min 1 tahun (Pegawai (non-manajemen & non-supe...,Indonesia - Jakarta Raya,[SEP]Candidate must possess at least Diploma i...,11 hari,Pendidikan,51 - 200 pekerja,SABTU-KAMIS,Formil (contoh: Kemeja + Dasi),Kesehatan,Bahasa Indonesia
4,jobstreet,Field Collection (JS - FC) Pati,OTO Group,Lulusan baru/Pengalaman kerja kurang dari 1 tahun,Indonesia - Jawa Tengah - Pati,[SEP]Lingkup pekerjaan: [SEP]Menjaga account d...,Lebih dari 2 minggu,Perbankan/Pelayanan Keuangan,Lebih dari 5000 pekerja,"Waktu regular, Senin - Jumat",Bisnis (contoh: Kemeja),"Kesehatan, Gigi, bonus tahunan",Bahasa Indonesia


# Data cleaning
Terdapat beberapa hal yang dilakukan:

A. Common cleaning
1. Convert to lowercase
2. Fillna dengan string 'Tidak tersedia'

B. Ekstrak data

1. <code>job_position</code>: dilakukan pengelompokkan/cluster supaya lebih unique menjadi job type.
    - Example: 'sales executive veteriner', 'corporate sales executive' menjadi -> 'sales executive'
2. <code>company_location</code>: dapat diekstrak menjadi attribut <code>pulau</code> dan <code>provinsi</code>
3. <code>job_description</code>: dapat diekstrak menjadi atribut <code>skill</code> dan <code>pendidikan</code>

In [ ]:
data.describe().T

,count,unique,top,freq
platform,14150,1,jobstreet,14150
job_position,14141,10582,Sales Executive,60
company_name,14141,5106,Perusahaan Dirahasiakan,548
years_of_experience,14137,77,Min 1 tahun (Pegawai (non-manajemen & non-supe...,3417
company_location,12720,3111,Indonesia - Jakarta Raya,1786
job_description,14141,13907,Dicari Marketing Freelance bidang Property Kua...,10
average_processing_time,13171,14,Lebih dari 2 minggu,10089
company_industry,13593,58,Manufaktur/Produksi,1712
company_size,13579,8,51 - 200 pekerja,3578
work_environment_waktu_bekerja,12519,715,"Waktu regular, Senin - Jumat",7999


In [ ]:
# check missing values
data.isnull().sum()

platform                               0
job_position                           9
company_name                           9
years_of_experience                   13
company_location                    1430
job_description                        9
average_processing_time              979
company_industry                     557
company_size                         571
work_environment_waktu_bekerja      1631
work_environment_gaya_berpakaian    1720
work_environment_tunjangan          2724
work_environment_bahasa             1659
dtype: int64

## Dropna dan duplicate
- Lakukan dropna() pada row yang kosong berdasarkan job_position (9 data).

In [ ]:
data = data.dropna(subset=['job_position'])
data.isnull().sum()

platform                               0
job_position                           0
company_name                           0
years_of_experience                    4
company_location                    1421
job_description                        0
average_processing_time              970
company_industry                     548
company_size                         562
work_environment_waktu_bekerja      1622
work_environment_gaya_berpakaian    1711
work_environment_tunjangan          2715
work_environment_bahasa             1650
dtype: int64

In [ ]:
# check duplicate
duplicate = data[data.duplicated()]
duplicate

,platform,job_position,company_name,years_of_experience,company_location,job_description,average_processing_time,company_industry,company_size,work_environment_waktu_bekerja,work_environment_gaya_berpakaian,work_environment_tunjangan,work_environment_bahasa
193,jobstreet,Warehouse Superintendent,PT Matahari Putra Makmur,Min 5 tahun (Manajer/Asisten Manajer),Indonesia - Jawa Timur - Pasuruan - GEMPOL,Uraian Pekerjaan :[SEP][SEP][SEP][SEP][SEP]Mer...,Lebih dari 2 minggu,Manufaktur/Produksi,501 - 1000 pekerja,Mondays-Saturdays,Bisnis (contoh: Kemeja),Kesehatan,Bahasa Indonesia
2384,jobstreet,Administration Staff,PT Relasi Abadi,Min 1 tahun (Pegawai (non-manajemen & non-supe...,Indonesia - Jakarta Raya - Jakarta Utara - Pan...,"[SEP][SEP]Preparing (extract, correct, sort, e...",Lebih dari 2 minggu,Konsultasi (Bisnis & Manajemen),1- 50 pekerja,"Waktu regular, Senin - Jumat",Kasual (contoh: Kaos),Parkir,Bahasa Indonesia
2940,jobstreet,INTERNET DATA CENTER TECHNICIAN,PT Cakra Persada Utama,Min 1 tahun (Pegawai (non-manajemen & non-supe...,Indonesia - Banten - Tangerang - SERPONG,[SEP]Installing servers and networking equipme...,1 hari,Komputer/Teknik Informatika (Perangkat Keras),1- 50 pekerja,"Waktu regular, Senin - Jumat",Kasual (contoh: Kaos),NaN,Bahasa Indonesia
3065,jobstreet,Insurance Specialist (Jakarta),PT PFI Mega Life Insurance,Min 1 tahun (Pegawai (non-manajemen & non-supe...,NaN,[SEP]Initiate and bridge cooperation with Part...,Lebih dari 2 minggu,Asuransi,51 - 200 pekerja,"Waktu regular, Senin - Jumat",Bisnis (contoh: Kemeja),Kesehatan,Bahasa Indonesia
3200,jobstreet,SAP Business One Technical Consultant,"PT. Metrodata Electronics, Tbk",Min 2 tahun (Pegawai (non-manajemen & non-supe...,Indonesia - Jakarta Raya - Jakarta Barat,[SEP]Job Description:[SEP][SEP][SEP]Melakukan ...,Lebih dari 2 minggu,"Konsultasi (IT, Ilmu Pengetahuan, Teknis & Tek...",1001 - 2000 pekerja,"Waktu regular, Senin - Jumat",Bisnis (contoh: Kemeja),"Kesehatan, Parkir",Bahasa Indonesia
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13552,jobstreet,Sales Executive,Sejati Stationery Co.,Min 1 tahun (Pegawai (non-manajemen & non-supe...,Indonesia - Jakarta Raya,"[SEP]Candidate must possess at least SMU, Dipl...",6 hari,Periklanan/Marketing/Promosi/Hubungan Masyarakat,1- 50 pekerja,"Waktu regular, Senin - Jumat",Bisnis (contoh: Kemeja),Kesehatan,Bahasa Indonesia
13722,jobstreet,Merchandising Manager,Kurnia Ciptamoda Gemilang,Min 3 tahun (Manajer/Asisten Manajer),Indonesia - Jakarta Raya - Jakarta Selatan - J...,"[SEP][SEP]Reporting to Brand Manager, you will...",1 hari,Retail/Merchandise,501 - 1000 pekerja,"Mon - Friday, Flexible working hours",Smart Casual,"Kesehatan, Tunjangan Lainnya, BPJS",Bahasa Indonesia and English
13912,jobstreet,BUSSINESS DEVELOPMENT MANAGER / CLIENT SERVICING,PT Spire Indonesia,Min 3 tahun (Manajer/Asisten Manajer),Indonesia - Jakarta Raya - Jl Jend Sudirman ka...,[SEP][SEP][SEP]Job Scope and Roles[SEP][SEP]De...,Lebih dari 2 minggu,Konsultasi (Bisnis & Manajemen),1- 50 pekerja,"Waktu regular, Senin - Jumat",Bisnis (contoh: Kemeja),"Kesehatan, Tunjangan Lainnya, Bantuan Pendidikan",Bahasa Inggris
13927,jobstreet,Accounting & General Admin,PT Berkat Jaya Damai,Min 3 tahun (Pegawai (non-manajemen & non-supe...,Indonesia - Jakarta Raya - Jakarta Utara - Kam...,a. Usia maksimal 35 tahun [SEP]b. Pendidikan m...,Lebih dari 2 minggu,Makanan & Minuman/Katering/Restoran,1- 50 pekerja,"Waktu regular, Senin - Jumat",Kasual (contoh: Kaos),NaN,Bahasa Indonesia


In [ ]:
# drop duplicate data
data = data.drop_duplicates().reset_index(drop=True)

# check duplicate lagi
len(data[data.duplicated()])

0

In [ ]:
data.describe().T

,count,unique,top,freq
platform,14059,1,jobstreet,14059
job_position,14059,10582,Sales Executive,59
company_name,14059,5106,Perusahaan Dirahasiakan,545
years_of_experience,14055,77,Min 1 tahun (Pegawai (non-manajemen & non-supe...,3401
company_location,12648,3111,Indonesia - Jakarta Raya,1778
job_description,14059,13907,Dicari Marketing Freelance bidang Property Kua...,10
average_processing_time,13096,14,Lebih dari 2 minggu,10031
company_industry,13514,58,Manufaktur/Produksi,1697
company_size,13500,8,51 - 200 pekerja,3555
work_environment_waktu_bekerja,12444,715,"Waktu regular, Senin - Jumat",7952


## Common cleaning

### company_name

In [ ]:
# check null values
data['company_name'].isnull().sum()

0

In [ ]:
data['company_name'].value_counts()

Perusahaan Dirahasiakan                                       545
OTO Group                                                     142
PT Michael Page Internasional Indonesia (Recruitment Firm)     83
PT. Aurora Group (Recruitment Firm)                            62
PT AXA Mandiri Financial Services                              61
                                                             ... 
PT Batavia Wine International                                   1
PT Artha Jasakonsulindo                                         1
PT Container Maritime Activities                                1
LOCETTE                                                         1
CV Indoganic Karyatama                                          1
Name: company_name, Length: 5106, dtype: int64

> tidak dilakukan cleaning, karena sudah cukup bagus

In [ ]:
data.head()

,platform,job_position,company_name,years_of_experience,company_location,job_description,average_processing_time,company_industry,company_size,work_environment_waktu_bekerja,work_environment_gaya_berpakaian,work_environment_tunjangan,work_environment_bahasa
0,jobstreet,SPV FINANCE,PT Axindo Infotama,Min 3 tahun (Supervisor/Koordinator),Indonesia - Jakarta Raya - Jakarta Pusat - Ruk...,[SEP][SEP]Usia Max. 35 tahun[SEP]Pendidikan Mi...,6 hari,Retail/Merchandise,51 - 200 pekerja,Sabtu atau memerlukan shift,Formil (contoh: Kemeja + Dasi),"Kesehatan, Parkir",Bahasa Indonesia
1,jobstreet,Qualitative Research Executive,Perusahaan Dirahasiakan,Min 3 tahun (Pegawai (non-manajemen & non-supe...,Indonesia - Jakarta Raya,RESPONSIBILITIES:[SEP]Brief Description of Dut...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,jobstreet,Field Collection (JS - FC) Wonogiri,OTO Group,Lulusan baru/Pengalaman kerja kurang dari 1 tahun,Indonesia - Jawa Tengah - Wonogiri Wonogoro - ...,[SEP]Lingkup pekerjaan: [SEP]Menjaga account d...,Lebih dari 2 minggu,Perbankan/Pelayanan Keuangan,Lebih dari 5000 pekerja,"Waktu regular, Senin - Jumat",Bisnis (contoh: Kemeja),"Kesehatan, Gigi, bonus tahunan",Bahasa Indonesia
3,jobstreet,Perawat di Negara Jerman,PT Jerman Manajemen,Min 1 tahun (Pegawai (non-manajemen & non-supe...,Indonesia - Jakarta Raya,[SEP]Candidate must possess at least Diploma i...,11 hari,Pendidikan,51 - 200 pekerja,SABTU-KAMIS,Formil (contoh: Kemeja + Dasi),Kesehatan,Bahasa Indonesia
4,jobstreet,Field Collection (JS - FC) Pati,OTO Group,Lulusan baru/Pengalaman kerja kurang dari 1 tahun,Indonesia - Jawa Tengah - Pati,[SEP]Lingkup pekerjaan: [SEP]Menjaga account d...,Lebih dari 2 minggu,Perbankan/Pelayanan Keuangan,Lebih dari 5000 pekerja,"Waktu regular, Senin - Jumat",Bisnis (contoh: Kemeja),"Kesehatan, Gigi, bonus tahunan",Bahasa Indonesia


### years_of_experience

> **Question**: Apakah perlu dihilangkan teks seperti '(pegawai (non-manajemen & non-supervisor))' supaya data lebih distinct? **Iya**

In [ ]:
# check null
data['years_of_experience'].isnull().sum()

4

In [ ]:
# years_experience
# convert to lower case dan fillna dengan string 'tidak tersedia'
yoe = data['years_of_experience'].str.lower().fillna('kosong')

yoe.value_counts()

min 1 tahun (pegawai (non-manajemen & non-supervisor))    3401
min 2 tahun (pegawai (non-manajemen & non-supervisor))    2448
min 3 tahun (pegawai (non-manajemen & non-supervisor))    1054
min 2 tahun (supervisor/koordinator)                       931
min 3 tahun (supervisor/koordinator)                       902
                                                          ... 
min 8 tahun (senior manager)                                 1
min 11 tahun (manajer/asisten manajer)                       1
min 5 tahun (manager)                                        1
min 3 tahun (senior executive)                               1
min 14 tahun (ceo/gm/direktur/manajer senior)                1
Name: years_of_experience, Length: 78, dtype: int64

In [ ]:
# coba lihat distinct data
temp = yoe.values
for idx, years in enumerate(temp):
    temp[idx] = years.split('(')[0].strip()

pd.Series(temp).value_counts()

min 1 tahun                                          4137
min 2 tahun                                          3743
min 3 tahun                                          2509
min 5 tahun                                          1665
lulusan baru/pengalaman kerja kurang dari 1 tahun     786
min 4 tahun                                           439
min 10 tahun                                          268
min 8 tahun                                           150
min 7 tahun                                           135
min 6 tahun                                           100
min 15 tahun                                           63
min 12 tahun                                           24
min 9 tahun                                             9
min 13 tahun                                            4
supervisor/koordinator                                  4
kosong                                                  4
pegawai                                                 3
min 16 tahun  

In [ ]:
# mapping data
yoe_mapper = {"lulusan baru/pengalaman kerja kurang dari 1 tahun": 0,
              "min 1 tahun": 1,
              "min 2 tahun": 2,
              "min 3 tahun": 3,
              "min 4 tahun": 4,
              "min 5 tahun": 5,
              "min 6 tahun": 6,
              "min 7 tahun": 7,
              "min 8 tahun": 8,
              "min 9 tahun": 9,
              "min 10 tahun": 10,
              "min 11 tahun": 11,
              "min 12 tahun": 12,
              "min 13 tahun": 13,
              "min 14 tahun": 14,
              "min 15 tahun": 15,
              "min 16 tahun": 16,
              "min 17 tahun": 17,
              "min 18 tahun": 18,
              "min 20 tahun": 20,
              "lebih dari 24 tahun": 24,
              "kosong": "tidak dicantumkan",
              "supervisor/koordinator": "tidak dicantumkan",
              "pegawai": "tidak dicantumkan",
              "ceo/gm/direktur/manajer senior": "tidak dicantumkan",
              "entry level": "tidak dicantumkan",
              "manajer/asisten manajer": "tidak dicantumkan"}

temp = pd.Series(temp).map(yoe_mapper)

In [ ]:
data['years_of_experience'] = temp
data['years_of_experience'].value_counts()

1                    4137
2                    3743
3                    2509
5                    1665
0                     786
4                     439
10                    268
8                     150
7                     135
6                     100
15                     63
12                     24
tidak dicantumkan      15
9                       9
13                      4
24                      2
16                      2
17                      2
18                      2
20                      2
14                      1
11                      1
Name: years_of_experience, dtype: int64

In [ ]:
data.head()

,platform,job_position,company_name,years_of_experience,company_location,job_description,average_processing_time,company_industry,company_size,work_environment_waktu_bekerja,work_environment_gaya_berpakaian,work_environment_tunjangan,work_environment_bahasa
0,jobstreet,SPV FINANCE,PT Axindo Infotama,3,Indonesia - Jakarta Raya - Jakarta Pusat - Ruk...,[SEP][SEP]Usia Max. 35 tahun[SEP]Pendidikan Mi...,6 hari,Retail/Merchandise,51 - 200 pekerja,Sabtu atau memerlukan shift,Formil (contoh: Kemeja + Dasi),"Kesehatan, Parkir",Bahasa Indonesia
1,jobstreet,Qualitative Research Executive,Perusahaan Dirahasiakan,3,Indonesia - Jakarta Raya,RESPONSIBILITIES:[SEP]Brief Description of Dut...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,jobstreet,Field Collection (JS - FC) Wonogiri,OTO Group,0,Indonesia - Jawa Tengah - Wonogiri Wonogoro - ...,[SEP]Lingkup pekerjaan: [SEP]Menjaga account d...,Lebih dari 2 minggu,Perbankan/Pelayanan Keuangan,Lebih dari 5000 pekerja,"Waktu regular, Senin - Jumat",Bisnis (contoh: Kemeja),"Kesehatan, Gigi, bonus tahunan",Bahasa Indonesia
3,jobstreet,Perawat di Negara Jerman,PT Jerman Manajemen,1,Indonesia - Jakarta Raya,[SEP]Candidate must possess at least Diploma i...,11 hari,Pendidikan,51 - 200 pekerja,SABTU-KAMIS,Formil (contoh: Kemeja + Dasi),Kesehatan,Bahasa Indonesia
4,jobstreet,Field Collection (JS - FC) Pati,OTO Group,0,Indonesia - Jawa Tengah - Pati,[SEP]Lingkup pekerjaan: [SEP]Menjaga account d...,Lebih dari 2 minggu,Perbankan/Pelayanan Keuangan,Lebih dari 5000 pekerja,"Waktu regular, Senin - Jumat",Bisnis (contoh: Kemeja),"Kesehatan, Gigi, bonus tahunan",Bahasa Indonesia


Dibuat categorical,
- 0 to 4 
- 5 to 9
- 10 to 14
- 15 to 19
- 20 to 24
- more than 24

In [ ]:
temp = data['years_of_experience'].copy()

for i in range(len(temp)):
    if temp[i] == 'tidak dicantumkan':
        continue

    year = temp[i]
    if year >= 0 and year <= 4:
        temp[i] = '0 - 4 tahun'
    elif year >= 5 and year <= 9:
        temp[i] = '5 - 9 tahun'
    elif year >= 10 and year <= 14:
        temp[i] = '10 - 14 tahun'
    elif year >= 15 and year <= 19:
        temp[i] = '15 - 19 tahun'
    elif year >= 20 and year <=24:
        temp[i] = '20 - 24 tahun'
    else:
        temp[i] = '> 24 tahun'

temp.value_counts()

0 - 4 tahun          11614
5 - 9 tahun           2059
10 - 14 tahun          298
15 - 19 tahun           69
tidak dicantumkan       15
20 - 24 tahun            4
Name: years_of_experience, dtype: int64

In [ ]:
# input data category years of experience into new column
data['years_of_experience_cat'] = temp
data['years_of_experience_cat'].isnull().sum()

0

In [ ]:
data.head()

,platform,job_position,company_name,years_of_experience,company_location,job_description,average_processing_time,company_industry,company_size,work_environment_waktu_bekerja,work_environment_gaya_berpakaian,work_environment_tunjangan,work_environment_bahasa,years_of_experience_cat
0,jobstreet,SPV FINANCE,PT Axindo Infotama,3,Indonesia - Jakarta Raya - Jakarta Pusat - Ruk...,[SEP][SEP]Usia Max. 35 tahun[SEP]Pendidikan Mi...,6 hari,Retail/Merchandise,51 - 200 pekerja,Sabtu atau memerlukan shift,Formil (contoh: Kemeja + Dasi),"Kesehatan, Parkir",Bahasa Indonesia,0 - 4 tahun
1,jobstreet,Qualitative Research Executive,Perusahaan Dirahasiakan,3,Indonesia - Jakarta Raya,RESPONSIBILITIES:[SEP]Brief Description of Dut...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 - 4 tahun
2,jobstreet,Field Collection (JS - FC) Wonogiri,OTO Group,0,Indonesia - Jawa Tengah - Wonogiri Wonogoro - ...,[SEP]Lingkup pekerjaan: [SEP]Menjaga account d...,Lebih dari 2 minggu,Perbankan/Pelayanan Keuangan,Lebih dari 5000 pekerja,"Waktu regular, Senin - Jumat",Bisnis (contoh: Kemeja),"Kesehatan, Gigi, bonus tahunan",Bahasa Indonesia,0 - 4 tahun
3,jobstreet,Perawat di Negara Jerman,PT Jerman Manajemen,1,Indonesia - Jakarta Raya,[SEP]Candidate must possess at least Diploma i...,11 hari,Pendidikan,51 - 200 pekerja,SABTU-KAMIS,Formil (contoh: Kemeja + Dasi),Kesehatan,Bahasa Indonesia,0 - 4 tahun
4,jobstreet,Field Collection (JS - FC) Pati,OTO Group,0,Indonesia - Jawa Tengah - Pati,[SEP]Lingkup pekerjaan: [SEP]Menjaga account d...,Lebih dari 2 minggu,Perbankan/Pelayanan Keuangan,Lebih dari 5000 pekerja,"Waktu regular, Senin - Jumat",Bisnis (contoh: Kemeja),"Kesehatan, Gigi, bonus tahunan",Bahasa Indonesia,0 - 4 tahun


### average_processing_time

In [ ]:
# convert to lower case dan fillna dengan string 'tidak tersedia'
avp = data['average_processing_time'].str.lower().fillna('tidak tersedia')

avp.value_counts()

lebih dari 2 minggu    10031
tidak tersedia           963
1 hari                   715
2 hari                   277
6 hari                   249
11 hari                  213
3 hari                   213
10 hari                  201
4 hari                   194
12 hari                  188
5 hari                   188
13 hari                  177
8 hari                   173
9 hari                   144
7 hari                   133
Name: average_processing_time, dtype: int64

In [ ]:
# disamakan satuan menjadi hari
temp = avp

for i in range(len(temp)):
    if temp[i] == 'tidak tersedia':
        continue
    
    if temp[i] == 'lebih dari 2 minggu':
        temp[i] = 14
    else:
        temp[i] = temp[i].split()[0]

temp.value_counts()

14                10031
tidak tersedia      963
1                   715
2                   277
6                   249
3                   213
11                  213
10                  201
4                   194
12                  188
5                   188
13                  177
8                   173
9                   144
7                   133
Name: average_processing_time, dtype: int64

In [ ]:
# check null values
data['average_processing_time'] = avp
data['average_processing_time'].isnull().sum()

0

In [ ]:
data.head()

,platform,job_position,company_name,years_of_experience,company_location,job_description,average_processing_time,company_industry,company_size,work_environment_waktu_bekerja,work_environment_gaya_berpakaian,work_environment_tunjangan,work_environment_bahasa,years_of_experience_cat
0,jobstreet,SPV FINANCE,PT Axindo Infotama,3,Indonesia - Jakarta Raya - Jakarta Pusat - Ruk...,[SEP][SEP]Usia Max. 35 tahun[SEP]Pendidikan Mi...,6,Retail/Merchandise,51 - 200 pekerja,Sabtu atau memerlukan shift,Formil (contoh: Kemeja + Dasi),"Kesehatan, Parkir",Bahasa Indonesia,0 - 4 tahun
1,jobstreet,Qualitative Research Executive,Perusahaan Dirahasiakan,3,Indonesia - Jakarta Raya,RESPONSIBILITIES:[SEP]Brief Description of Dut...,tidak tersedia,NaN,NaN,NaN,NaN,NaN,NaN,0 - 4 tahun
2,jobstreet,Field Collection (JS - FC) Wonogiri,OTO Group,0,Indonesia - Jawa Tengah - Wonogiri Wonogoro - ...,[SEP]Lingkup pekerjaan: [SEP]Menjaga account d...,14,Perbankan/Pelayanan Keuangan,Lebih dari 5000 pekerja,"Waktu regular, Senin - Jumat",Bisnis (contoh: Kemeja),"Kesehatan, Gigi, bonus tahunan",Bahasa Indonesia,0 - 4 tahun
3,jobstreet,Perawat di Negara Jerman,PT Jerman Manajemen,1,Indonesia - Jakarta Raya,[SEP]Candidate must possess at least Diploma i...,11,Pendidikan,51 - 200 pekerja,SABTU-KAMIS,Formil (contoh: Kemeja + Dasi),Kesehatan,Bahasa Indonesia,0 - 4 tahun
4,jobstreet,Field Collection (JS - FC) Pati,OTO Group,0,Indonesia - Jawa Tengah - Pati,[SEP]Lingkup pekerjaan: [SEP]Menjaga account d...,14,Perbankan/Pelayanan Keuangan,Lebih dari 5000 pekerja,"Waktu regular, Senin - Jumat",Bisnis (contoh: Kemeja),"Kesehatan, Gigi, bonus tahunan",Bahasa Indonesia,0 - 4 tahun


### company_industry

In [ ]:
# convert to lower case dan fillna dengan string 'tidak tersedia'
ci = data['company_industry'].fillna('tidak tersedia')

ci.value_counts()

Manufaktur/Produksi                                     1697
Perbankan/Pelayanan Keuangan                             882
Komputer/Teknik Informatika (Perangkat Lunak)            723
Umum & Grosir                                            700
Retail/Merchandise                                       684
Produk Konsumen/Barang konsumen yang bergerak cepat      617
Manajemen/Konsulting HR                                  613
tidak tersedia                                           545
Transportasi/Logistik                                    532
Properti/Real Estate                                     513
Makanan & Minuman/Katering/Restoran                      482
Konstruksi/Bangunan/Teknik                               482
Kesehatan/Medis                                          475
Agrikultural/Perkebunan/Peternakan Unggas/Perikanan      397
Konsultasi (IT, Ilmu Pengetahuan, Teknis & Teknikal)     377
Asuransi                                                 357
Telekomunikasi          

In [ ]:
# check null values
data['company_industry'] = ci
data['company_industry'].isnull().sum()

0

In [ ]:
data.head()

,platform,job_position,company_name,years_of_experience,company_location,job_description,average_processing_time,company_industry,company_size,work_environment_waktu_bekerja,work_environment_gaya_berpakaian,work_environment_tunjangan,work_environment_bahasa,years_of_experience_cat
0,jobstreet,SPV FINANCE,PT Axindo Infotama,3,Indonesia - Jakarta Raya - Jakarta Pusat - Ruk...,[SEP][SEP]Usia Max. 35 tahun[SEP]Pendidikan Mi...,6,Retail/Merchandise,51 - 200 pekerja,Sabtu atau memerlukan shift,Formil (contoh: Kemeja + Dasi),"Kesehatan, Parkir",Bahasa Indonesia,0 - 4 tahun
1,jobstreet,Qualitative Research Executive,Perusahaan Dirahasiakan,3,Indonesia - Jakarta Raya,RESPONSIBILITIES:[SEP]Brief Description of Dut...,tidak tersedia,tidak tersedia,NaN,NaN,NaN,NaN,NaN,0 - 4 tahun
2,jobstreet,Field Collection (JS - FC) Wonogiri,OTO Group,0,Indonesia - Jawa Tengah - Wonogiri Wonogoro - ...,[SEP]Lingkup pekerjaan: [SEP]Menjaga account d...,14,Perbankan/Pelayanan Keuangan,Lebih dari 5000 pekerja,"Waktu regular, Senin - Jumat",Bisnis (contoh: Kemeja),"Kesehatan, Gigi, bonus tahunan",Bahasa Indonesia,0 - 4 tahun
3,jobstreet,Perawat di Negara Jerman,PT Jerman Manajemen,1,Indonesia - Jakarta Raya,[SEP]Candidate must possess at least Diploma i...,11,Pendidikan,51 - 200 pekerja,SABTU-KAMIS,Formil (contoh: Kemeja + Dasi),Kesehatan,Bahasa Indonesia,0 - 4 tahun
4,jobstreet,Field Collection (JS - FC) Pati,OTO Group,0,Indonesia - Jawa Tengah - Pati,[SEP]Lingkup pekerjaan: [SEP]Menjaga account d...,14,Perbankan/Pelayanan Keuangan,Lebih dari 5000 pekerja,"Waktu regular, Senin - Jumat",Bisnis (contoh: Kemeja),"Kesehatan, Gigi, bonus tahunan",Bahasa Indonesia,0 - 4 tahun


### company_size

In [ ]:
# convert to lower case dan fillna dengan string 'tidak spesifik'
cs = data['company_size'].str.lower().fillna('tidak spesifik')

cs.value_counts()

51 - 200 pekerja           3555
1- 50 pekerja              2731
201 - 500 pekerja          2149
501 - 1000 pekerja         1501
1001 - 2000 pekerja        1227
lebih dari 5000 pekerja    1157
2001 - 5000 pekerja         944
tidak spesifik              795
Name: company_size, dtype: int64

In [ ]:
# check null values
data['company_size'] = cs
data['company_size'].isnull().sum()

0

In [ ]:
data.head()

,platform,job_position,company_name,years_of_experience,company_location,job_description,average_processing_time,company_industry,company_size,work_environment_waktu_bekerja,work_environment_gaya_berpakaian,work_environment_tunjangan,work_environment_bahasa,years_of_experience_cat
0,jobstreet,SPV FINANCE,PT Axindo Infotama,3,Indonesia - Jakarta Raya - Jakarta Pusat - Ruk...,[SEP][SEP]Usia Max. 35 tahun[SEP]Pendidikan Mi...,6,Retail/Merchandise,51 - 200 pekerja,Sabtu atau memerlukan shift,Formil (contoh: Kemeja + Dasi),"Kesehatan, Parkir",Bahasa Indonesia,0 - 4 tahun
1,jobstreet,Qualitative Research Executive,Perusahaan Dirahasiakan,3,Indonesia - Jakarta Raya,RESPONSIBILITIES:[SEP]Brief Description of Dut...,tidak tersedia,tidak tersedia,tidak spesifik,NaN,NaN,NaN,NaN,0 - 4 tahun
2,jobstreet,Field Collection (JS - FC) Wonogiri,OTO Group,0,Indonesia - Jawa Tengah - Wonogiri Wonogoro - ...,[SEP]Lingkup pekerjaan: [SEP]Menjaga account d...,14,Perbankan/Pelayanan Keuangan,lebih dari 5000 pekerja,"Waktu regular, Senin - Jumat",Bisnis (contoh: Kemeja),"Kesehatan, Gigi, bonus tahunan",Bahasa Indonesia,0 - 4 tahun
3,jobstreet,Perawat di Negara Jerman,PT Jerman Manajemen,1,Indonesia - Jakarta Raya,[SEP]Candidate must possess at least Diploma i...,11,Pendidikan,51 - 200 pekerja,SABTU-KAMIS,Formil (contoh: Kemeja + Dasi),Kesehatan,Bahasa Indonesia,0 - 4 tahun
4,jobstreet,Field Collection (JS - FC) Pati,OTO Group,0,Indonesia - Jawa Tengah - Pati,[SEP]Lingkup pekerjaan: [SEP]Menjaga account d...,14,Perbankan/Pelayanan Keuangan,lebih dari 5000 pekerja,"Waktu regular, Senin - Jumat",Bisnis (contoh: Kemeja),"Kesehatan, Gigi, bonus tahunan",Bahasa Indonesia,0 - 4 tahun


### work_environment_waktu_bekerja
dapat dikelompokkan
- senin - jum'at
- senin - sabtu
- flexible
- shift

In [ ]:
# convert to lower case dan fillna dengan string 'tidak tersedia'
waktu_bekerja = data['work_environment_waktu_bekerja'].str.lower().fillna('tidak tersedia')

waktu_bekerja.value_counts()

waktu regular, senin - jumat                                                      7952
tidak tersedia                                                                    1615
sabtu atau memerlukan shift                                                       1127
senin - sabtu                                                                      353
monday - saturday                                                                  268
                                                                                  ... 
senin (08.00-16.30) sabtu (08.00-14.30). 2 shift jika diperlukan (13.30-22.00)       1
monday to saturday (half-day)                                                        1
hari senin - sabtu,                                                                  1
work is our life                                                                     1
shifting, 1 hari libur dalam satu minggu, dan libur pada public holiday              1
Name: work_environment_waktu_bekerja, Lengt

In [ ]:
c = pd.read_csv('/content/drive/My Drive/Data Loker/waktu_bekerja_manual_perbaikan.csv')
c['work_environment_waktu_bekerja'].value_counts()

senin - jumat      8555
senin - sabtu      2219
tidak tersedia     1621
sabtu              1132
tidak spesifik      339
shift                71
fleksibel            56
senin - minggu       34
roster               25
selasa - minggu       4
minggu - jumat        2
sabtu - kamis         1
Name: work_environment_waktu_bekerja, dtype: int64

ganti roster -> shift

In [ ]:
c['work_environment_waktu_bekerja'] = c['work_environment_waktu_bekerja'].str.replace('roster', 'shift', regex=False)
c['work_environment_waktu_bekerja'].value_counts()

senin - jumat      8555
senin - sabtu      2219
tidak tersedia     1621
sabtu              1132
tidak spesifik      339
shift                96
fleksibel            56
senin - minggu       34
selasa - minggu       4
minggu - jumat        2
sabtu - kamis         1
Name: work_environment_waktu_bekerja, dtype: int64

In [ ]:
# check null values
data['work_environment_waktu_bekerja'] = c['work_environment_waktu_bekerja']
data['work_environment_waktu_bekerja'].isnull().sum()

0

In [ ]:
data.head()

,platform,job_position,company_name,years_of_experience,company_location,job_description,average_processing_time,company_industry,company_size,work_environment_waktu_bekerja,work_environment_gaya_berpakaian,work_environment_tunjangan,work_environment_bahasa,years_of_experience_cat
0,jobstreet,SPV FINANCE,PT Axindo Infotama,3,Indonesia - Jakarta Raya - Jakarta Pusat - Ruk...,[SEP][SEP]Usia Max. 35 tahun[SEP]Pendidikan Mi...,6,Retail/Merchandise,51 - 200 pekerja,sabtu,Formil (contoh: Kemeja + Dasi),"Kesehatan, Parkir",Bahasa Indonesia,0 - 4 tahun
1,jobstreet,Qualitative Research Executive,Perusahaan Dirahasiakan,3,Indonesia - Jakarta Raya,RESPONSIBILITIES:[SEP]Brief Description of Dut...,tidak tersedia,tidak tersedia,tidak spesifik,tidak tersedia,NaN,NaN,NaN,0 - 4 tahun
2,jobstreet,Field Collection (JS - FC) Wonogiri,OTO Group,0,Indonesia - Jawa Tengah - Wonogiri Wonogoro - ...,[SEP]Lingkup pekerjaan: [SEP]Menjaga account d...,14,Perbankan/Pelayanan Keuangan,lebih dari 5000 pekerja,senin - jumat,Bisnis (contoh: Kemeja),"Kesehatan, Gigi, bonus tahunan",Bahasa Indonesia,0 - 4 tahun
3,jobstreet,Perawat di Negara Jerman,PT Jerman Manajemen,1,Indonesia - Jakarta Raya,[SEP]Candidate must possess at least Diploma i...,11,Pendidikan,51 - 200 pekerja,sabtu - kamis,Formil (contoh: Kemeja + Dasi),Kesehatan,Bahasa Indonesia,0 - 4 tahun
4,jobstreet,Field Collection (JS - FC) Pati,OTO Group,0,Indonesia - Jawa Tengah - Pati,[SEP]Lingkup pekerjaan: [SEP]Menjaga account d...,14,Perbankan/Pelayanan Keuangan,lebih dari 5000 pekerja,senin - jumat,Bisnis (contoh: Kemeja),"Kesehatan, Gigi, bonus tahunan",Bahasa Indonesia,0 - 4 tahun


### work_environment_gaya_berpakaian
Dilakukan pengelompokkan
- bisnis
- kasual
- kasual cerdas
- kasual bisnis
- formal
- semi formal
- seragam
- seragam lapangan
- modis
- pakaian muslim
- tidak spesifik
- tidak tersedia

In [ ]:
# convert to lower case dan fillna dengan string 'tidak tersedia'
gaya_berpakaian = data['work_environment_gaya_berpakaian'].str.lower().fillna('tidak tersedia')

gaya_berpakaian.value_counts()

bisnis (contoh: kemeja)           7001
kasual (contoh: kaos)             2283
tidak tersedia                    1704
formil (contoh: kemeja + dasi)    1538
uniform                            307
                                  ... 
smart casual bussiness               1
bebas, rapih dan sopan               1
seragam standar perusahaan           1
xlc uniform                          1
official (formal)                    1
Name: work_environment_gaya_berpakaian, Length: 269, dtype: int64

In [ ]:
# gaya_berpakaian.value_counts().to_csv('gaya_berpakaian_distinct.csv')
# gaya_berpakaian.to_csv('gaya_berpakaian_kotor.csv')

Dari data distinct menggunakan .value_counts(), dilakukan manual mapping pada google sheet.

In [ ]:
c = pd.read_csv("/content/drive/My Drive/Data Loker/gaya_berpakaian_mapper.csv")

# sorting multiple data
for index, row in c.iterrows():
    values = row['value'].split(',')
    values = [data.strip() for data in values]
    result = sorted(values)

    # update
    row['value'] = ', '.join(result).strip()

# create dict for mapper
pakaian_mapper = dict()
for idx, row in c.iterrows():
    pakaian_mapper[row['key']] = row['value']

In [ ]:
temp = gaya_berpakaian.map(pakaian_mapper)
temp.value_counts()

bisnis                              7087
kasual                              2433
tidak tersedia                      1712
formal                              1664
seragam                              579
kasual cerdas                        159
bisnis, seragam                       82
kasual bisnis                         67
formal, kasual                        55
bisnis, kasual                        42
kasual, seragam                       30
formal, seragam                       20
kasual formal                         19
seragam lapangan                      18
semi formal                           16
kasual cerdas, seragam                12
pakaian muslim                        12
tidak spesifik                        12
bisnis, formal, seragam lapangan      10
bisnis, formal                        10
formal, modis                          7
kasual, kasual bisnis                  3
bisnis, seragam lapangan               2
bisnis, kasual, seragam                1
kasual, semi for

Lakukan replace:
- kasual cerdas -> semi formal
- kasual bisnis -> semi formal
- seragam lapangan -> seragam
- modis -> kasual
- tidak tersedia -> tidak spesifik

In [ ]:
style_mapper = {'bisnis': 'bisnis',
                'kasual': 'kasual',
                'kasual cerdas': 'kasual',
                'kasual bisnis': 'semi formal',
                'formal': 'formal',
                'semi formal': 'semi formal',
                'kasual formal': 'semi formal',
                'seragam': 'seragam',
                'seragam lapangan': 'seragam',
                'modis': 'kasual',
                'pakaian muslim': 'pakaian muslim',
                'olahraga': 'olahraga',
                'tidak spesifik': 'tidak spesifik',
                'tidak tersedia': 'tidak spesifik'}

In [ ]:
# style mapping
result = list()
i = 0
for style in temp:
    split = [data.strip() for data in style.split(',')]
    mapping = pd.Series(split).map(style_mapper).dropna().drop_duplicates()
    join = ', '.join(list(mapping)).strip()

    result.append(join)

In [ ]:
pd.Series(result).value_counts()

bisnis                     7087
kasual                     2594
tidak spesifik             1724
formal                     1664
seragam                     597
semi formal                 102
bisnis, seragam              84
formal, kasual               62
bisnis, kasual               44
kasual, seragam              42
formal, seragam              21
pakaian muslim               12
bisnis, formal               10
bisnis, formal, seragam      10
kasual, semi formal           4
bisnis, kasual, seragam       1
olahraga                      1
dtype: int64

In [ ]:
# check null values
data['work_environment_gaya_berpakaian'] = result
data['work_environment_gaya_berpakaian'].isnull().sum()

0

In [ ]:
data.head()

,platform,job_position,company_name,years_of_experience,company_location,job_description,average_processing_time,company_industry,company_size,work_environment_waktu_bekerja,work_environment_gaya_berpakaian,work_environment_tunjangan,work_environment_bahasa,years_of_experience_cat
0,jobstreet,SPV FINANCE,PT Axindo Infotama,3,Indonesia - Jakarta Raya - Jakarta Pusat - Ruk...,[SEP][SEP]Usia Max. 35 tahun[SEP]Pendidikan Mi...,6,Retail/Merchandise,51 - 200 pekerja,sabtu,formal,"Kesehatan, Parkir",Bahasa Indonesia,0 - 4 tahun
1,jobstreet,Qualitative Research Executive,Perusahaan Dirahasiakan,3,Indonesia - Jakarta Raya,RESPONSIBILITIES:[SEP]Brief Description of Dut...,tidak tersedia,tidak tersedia,tidak spesifik,tidak tersedia,tidak spesifik,NaN,NaN,0 - 4 tahun
2,jobstreet,Field Collection (JS - FC) Wonogiri,OTO Group,0,Indonesia - Jawa Tengah - Wonogiri Wonogoro - ...,[SEP]Lingkup pekerjaan: [SEP]Menjaga account d...,14,Perbankan/Pelayanan Keuangan,lebih dari 5000 pekerja,senin - jumat,bisnis,"Kesehatan, Gigi, bonus tahunan",Bahasa Indonesia,0 - 4 tahun
3,jobstreet,Perawat di Negara Jerman,PT Jerman Manajemen,1,Indonesia - Jakarta Raya,[SEP]Candidate must possess at least Diploma i...,11,Pendidikan,51 - 200 pekerja,sabtu - kamis,formal,Kesehatan,Bahasa Indonesia,0 - 4 tahun
4,jobstreet,Field Collection (JS - FC) Pati,OTO Group,0,Indonesia - Jawa Tengah - Pati,[SEP]Lingkup pekerjaan: [SEP]Menjaga account d...,14,Perbankan/Pelayanan Keuangan,lebih dari 5000 pekerja,senin - jumat,bisnis,"Kesehatan, Gigi, bonus tahunan",Bahasa Indonesia,0 - 4 tahun


Untuk mempermudah dipahami bagi jobseeker, berikan contoh dari gaya berpakaian:
- bisnis: kemeja polos
- kasual: kaos
- formal: kemeja polos & dasi
- semi formal: kemeja motif
- pakaian muslim
- seragam: seragam kantor
- olahraga: olahraga

In [ ]:
contoh_style_mapper = {'bisnis': 'kemeja',
                       'kasual': 'kaos',
                       'formal': 'kemeja polos & dasi',
                       'semi formal': 'kemeja motif',
                       'pakaian muslim': 'pakaian muslim',
                       'seragam': 'seragam kantor/lapangan',
                       'olahraga': 'olahraga',
                       'tidak spesifik': 'tidak spesifik'}

In [ ]:
# contoh style mapping
result_style = list()

for style in result:
    split = [data.strip() for data in style.split(',')]
    mapping = pd.Series(split).map(contoh_style_mapper).dropna().drop_duplicates()
    join = ', '.join(list(mapping)).strip()

    result_style.append(join)

In [ ]:
pd.Series(result_style).value_counts()

kemeja                                                  7087
kaos                                                    2594
tidak spesifik                                          1724
kemeja polos & dasi                                     1664
seragam kantor/lapangan                                  597
kemeja motif                                             102
kemeja, seragam kantor/lapangan                           84
kemeja polos & dasi, kaos                                 62
kemeja, kaos                                              44
kaos, seragam kantor/lapangan                             42
kemeja polos & dasi, seragam kantor/lapangan              21
pakaian muslim                                            12
kemeja, kemeja polos & dasi                               10
kemeja, kemeja polos & dasi, seragam kantor/lapangan      10
kaos, kemeja motif                                         4
kemeja, kaos, seragam kantor/lapangan                      1
olahraga                

In [ ]:
# check null values
data['work_environment_gaya_berpakaian_contoh'] = result_style
data['work_environment_gaya_berpakaian_contoh'].isnull().sum()

0

In [ ]:
data.head()

,platform,job_position,company_name,years_of_experience,company_location,job_description,average_processing_time,company_industry,company_size,work_environment_waktu_bekerja,work_environment_gaya_berpakaian,work_environment_tunjangan,work_environment_bahasa,years_of_experience_cat,work_environment_gaya_berpakaian_contoh
0,jobstreet,SPV FINANCE,PT Axindo Infotama,3,Indonesia - Jakarta Raya - Jakarta Pusat - Ruk...,[SEP][SEP]Usia Max. 35 tahun[SEP]Pendidikan Mi...,6,Retail/Merchandise,51 - 200 pekerja,sabtu,formal,"Kesehatan, Parkir",Bahasa Indonesia,0 - 4 tahun,kemeja polos & dasi
1,jobstreet,Qualitative Research Executive,Perusahaan Dirahasiakan,3,Indonesia - Jakarta Raya,RESPONSIBILITIES:[SEP]Brief Description of Dut...,tidak tersedia,tidak tersedia,tidak spesifik,tidak tersedia,tidak spesifik,NaN,NaN,0 - 4 tahun,tidak spesifik
2,jobstreet,Field Collection (JS - FC) Wonogiri,OTO Group,0,Indonesia - Jawa Tengah - Wonogiri Wonogoro - ...,[SEP]Lingkup pekerjaan: [SEP]Menjaga account d...,14,Perbankan/Pelayanan Keuangan,lebih dari 5000 pekerja,senin - jumat,bisnis,"Kesehatan, Gigi, bonus tahunan",Bahasa Indonesia,0 - 4 tahun,kemeja
3,jobstreet,Perawat di Negara Jerman,PT Jerman Manajemen,1,Indonesia - Jakarta Raya,[SEP]Candidate must possess at least Diploma i...,11,Pendidikan,51 - 200 pekerja,sabtu - kamis,formal,Kesehatan,Bahasa Indonesia,0 - 4 tahun,kemeja polos & dasi
4,jobstreet,Field Collection (JS - FC) Pati,OTO Group,0,Indonesia - Jawa Tengah - Pati,[SEP]Lingkup pekerjaan: [SEP]Menjaga account d...,14,Perbankan/Pelayanan Keuangan,lebih dari 5000 pekerja,senin - jumat,bisnis,"Kesehatan, Gigi, bonus tahunan",Bahasa Indonesia,0 - 4 tahun,kemeja


### work_environment_tunjangan

In [ ]:
# convert to lower case dan fillna dengan string 'tidak tersedia'
tunjangan = data['work_environment_tunjangan'].str.lower().fillna('tidak tersedia')

tunjangan.value_counts()

kesehatan                                                                                                2801
tidak tersedia                                                                                           2702
kesehatan, tunjangan lainnya                                                                              952
kesehatan, parkir                                                                                         569
tunjangan lainnya                                                                                         538
                                                                                                         ... 
kesehatan, tunjangan lainnya, pinjaman, gigi, parkir, maternity, glasses                                    1
kesehatan, parkir, jamsostek                                                                                1
kesehatan, tunjangan lainnya, pinjaman, olahraga (contoh: pusat kebugaran), jamsostek (jht, jkk, jkm)       1
kesehatan,

In [ ]:
# tunjangan.value_counts().to_csv('tunjangan_distinct.csv')
# gaya_berpakaian.to_csv('gaya_berpakaian_kotor.csv')

In [ ]:
temp = list()
for i in range(len(tunjangan)):
    split = [data.strip() for data in tunjangan[i].split(',')]
    temp += split

Lihat jenis tunjangan unique

In [ ]:
from collections import Counter
a = Counter(temp)
a
# pd.Series(a).to_csv('tunjangan_item_mapper.csv')

Counter({'': 119,
         '& bpjs kesehatan': 2,
         '(sesuai dengan peraturan perusahaan)': 4,
         '-': 194,
         '- allowance (jika capai target)': 3,
         '- bpjs kesehatan': 3,
         '- bpjs kesehatan / bpjs ketenagakerjaan / asuransi kesehatan': 2,
         '- employee loan': 4,
         '- gaji pokok -closing fee': 1,
         '- life insurance. - career path': 4,
         '- monthly salary': 4,
         '- s/d rp 10.000.000': 1,
         '-bonus target': 1,
         '-bpjs': 1,
         '-bpjs ketenagakerjaan & kesehatan': 14,
         '-gaji pokok': 1,
         '-shuttle bus': 2,
         '-uang bensin': 1,
         '-uang kerajinan': 1,
         '.': 1,
         '12 days of annual leave': 2,
         '14 days for annual leave per year': 1,
         '16 days annual leave': 2,
         '30% - 40% of burgreens product': 1,
         '5-2%': 2,
         'acara refreshing bulanan': 4,
         'accommodation': 1,
         'accomodation': 13,
         'agricultu

In [ ]:
temp = pd.Series(list(a))
temp[27] = 'kosong'
temp.head()

0         kesehatan
1            parkir
2    tidak tersedia
3              gigi
4     bonus tahunan
dtype: object

Data tunjangan dilakukan perbaikan manual dengan google sheet.

In [ ]:
t1 = pd.read_csv("/content/drive/My Drive/Data Loker/tunjangan_item_mapper2.csv")

t1['Jenis Tunjangan2'] = t1['Jenis Tunjangan2'].str.lower()
t1['Jenis Tunjangan2'].value_counts()

bonus                 138
umum                  106
bpjs-jamsostek         71
makan                  49
transportasi           46
                     ... 
beras                   1
flexible                1
asuransi-jamsostek      1
makanan                 1
makan-pendidikan        1
Name: Jenis Tunjangan2, Length: 79, dtype: int64

In [ ]:
# t['Jenis Tunjangan2'].value_counts().to_csv('tunjangan_distinct3.csv')

In [ ]:
# create dict mapper
tunjangan_mapper = dict()

for idx, row in t1.iterrows():
    tunjangan_mapper[row['Jenis Tunjangan dari data asli (Jangan dirubah)']] = row['Jenis Tunjangan2']

In [ ]:
# iterate
result = list()
for i in range(len(tunjangan)):
    split = [data.strip() for data in tunjangan[i].split(',')]
    mapping = pd.Series(split).map(tunjangan_mapper).dropna().drop_duplicates()
    join = ', '.join(list(mapping)).strip()
    result.append(join)

In [ ]:
pd.Series(result).value_counts()

kesehatan                                              3000
tidak tersedia                                         2836
kesehatan, umum                                        2578
umum                                                    868
kesehatan, umum, kacamata                               379
                                                       ... 
tidak tersedia, pajak, bonus, kesehatan, pendidikan       1
kesehatan, makan, umum, transportasi, komunikasi          1
kesehatan, umum, kacamata, transportasi                   1
pendidikan, wellnes, kacamata                             1
                                                          1
Length: 499, dtype: int64

Data masih terdapat yang kotor, dilakukan cleaning manual lagi menggunakan google sheet.

In [ ]:
t2 = pd.read_csv("/content/drive/My Drive/Data Loker/tunjangan_mapper3.csv")

t2['target'] = t2['target'].str.lower()

temp = list()
for idx, row in t2.iterrows():
    split = [data.strip() for data in row['target'].split(',')]
    temp += split

pd.Series(temp).value_counts()

makan                   13
asuransi                12
transportasi            11
bpjs kesehatan          10
thr                      9
bpjs                     8
bonus                    8
perumahan                5
pendidikan               5
kesehatan                5
flexible benefits        4
traveling                4
jht                      3
komunikasi               3
kacamata                 3
gigi                     2
jabatan                  2
pernikahan               1
saham                    1
lembur                   1
transport                1
umum                     1
negotiable               1
member koperasi          1
operasional              1
bpjs ketenagakerjaan     1
lainnya                  1
pensiun                  1
tidak tersedia           1
promosi                  1
daerah                   1
pajak                    1
lahiran                  1
keluarga                 1
cuti                     1
dtype: int64

In [ ]:
# create dict mapper
tunjangan_mapper2 = dict()

for idx, row in t2.iterrows():
    tunjangan_mapper2[row['asli']] = row['target']

In [ ]:
# iterate
result2 = list()
for i in range(len(result)):
    split = [data.strip() for data in result[i].split(',')]
    mapping = pd.Series(split).map(tunjangan_mapper2).dropna().drop_duplicates()
    join = ', '.join(list(mapping)).strip()
    result2.append(join)

In [ ]:
# check unique
temp = list()
for i in range(len(result2)):
    split = [data.strip() for data in result2[i].split(',')]
    temp += split

pd.Series(temp).value_counts()

kesehatan               9310
umum                    5980
tidak tersedia          2921
kacamata                1290
bpjs kesehatan          1248
pendidikan               946
bonus                    899
makan                    436
transportasi             335
thr                      202
asuransi                 197
komunikasi               185
jht                      129
pensiun                  124
traveling                 81
jabatan                   80
perumahan                 69
lembur                    48
bpjs                      47
transport                 40
cuti                      40
lahiran                   39
pernikahan                39
lainnya                   37
daerah                    33
flexible benefits         29
saham                     16
member koperasi           14
negotiable                13
operasional               10
pajak                      8
promosi                    7
gigi                       7
bpjs ketenagakerjaan       4
keluarga      

Terdapat data yang hanya berisi character titik '.', dilakukan replace dengan 'tidak tersedia'

In [ ]:
# data asli
tunjangan[12045]

'.'

In [ ]:
result3 = pd.Series(result2)

# cari indeks data kosong
result3[result3 == '']

12045    
dtype: object

In [ ]:
# replace kosong dengan 'tidak tersedia'
result3.iloc[12045] = 'tidak tersedia'

In [ ]:
# check unique
temp = list()
for i in range(len(result3)):
    split = [data.strip() for data in result3[i].split(',')]
    temp += split

pd.Series(temp).value_counts()

kesehatan               9310
umum                    5980
tidak tersedia          2922
kacamata                1290
bpjs kesehatan          1248
pendidikan               946
bonus                    899
makan                    436
transportasi             335
thr                      202
asuransi                 197
komunikasi               185
jht                      129
pensiun                  124
traveling                 81
jabatan                   80
perumahan                 69
lembur                    48
bpjs                      47
transport                 40
cuti                      40
pernikahan                39
lahiran                   39
lainnya                   37
daerah                    33
flexible benefits         29
saham                     16
member koperasi           14
negotiable                13
operasional               10
pajak                      8
promosi                    7
gigi                       7
bpjs ketenagakerjaan       4
keluarga      

In [ ]:
# check null values
data['work_environment_tunjangan'] = result3.tolist()
data['work_environment_tunjangan'].isnull().sum()

0

In [ ]:
data.head()

,platform,job_position,company_name,years_of_experience,company_location,job_description,average_processing_time,company_industry,company_size,work_environment_waktu_bekerja,work_environment_gaya_berpakaian,work_environment_tunjangan,work_environment_bahasa,years_of_experience_cat,work_environment_gaya_berpakaian_contoh
0,jobstreet,SPV FINANCE,PT Axindo Infotama,3,Indonesia - Jakarta Raya - Jakarta Pusat - Ruk...,[SEP][SEP]Usia Max. 35 tahun[SEP]Pendidikan Mi...,6,Retail/Merchandise,51 - 200 pekerja,sabtu,formal,"kesehatan, umum",Bahasa Indonesia,0 - 4 tahun,kemeja polos & dasi
1,jobstreet,Qualitative Research Executive,Perusahaan Dirahasiakan,3,Indonesia - Jakarta Raya,RESPONSIBILITIES:[SEP]Brief Description of Dut...,tidak tersedia,tidak tersedia,tidak spesifik,tidak tersedia,tidak spesifik,tidak tersedia,NaN,0 - 4 tahun,tidak spesifik
2,jobstreet,Field Collection (JS - FC) Wonogiri,OTO Group,0,Indonesia - Jawa Tengah - Wonogiri Wonogoro - ...,[SEP]Lingkup pekerjaan: [SEP]Menjaga account d...,14,Perbankan/Pelayanan Keuangan,lebih dari 5000 pekerja,senin - jumat,bisnis,"kesehatan, bonus",Bahasa Indonesia,0 - 4 tahun,kemeja
3,jobstreet,Perawat di Negara Jerman,PT Jerman Manajemen,1,Indonesia - Jakarta Raya,[SEP]Candidate must possess at least Diploma i...,11,Pendidikan,51 - 200 pekerja,sabtu - kamis,formal,kesehatan,Bahasa Indonesia,0 - 4 tahun,kemeja polos & dasi
4,jobstreet,Field Collection (JS - FC) Pati,OTO Group,0,Indonesia - Jawa Tengah - Pati,[SEP]Lingkup pekerjaan: [SEP]Menjaga account d...,14,Perbankan/Pelayanan Keuangan,lebih dari 5000 pekerja,senin - jumat,bisnis,"kesehatan, bonus",Bahasa Indonesia,0 - 4 tahun,kemeja


### work_environment_bahasa

In [ ]:
# convert to lower case dan fillna dengan string 'tidak tersedia'
bahasa = data['work_environment_bahasa'].str.lower().fillna('tidak tersedia')

bahasa.value_counts()

bahasa indonesia                      9162
bahasa inggris                        1856
tidak tersedia                        1642
bahasa indonesia & english             208
bahasa indonesia - english             162
                                      ... 
inggris, indonesia, mandarin             1
indonesia and mandarin                   1
indonesia-inggris                        1
english, chinese, bahasa indonesia       1
english and korea                        1
Name: work_environment_bahasa, Length: 150, dtype: int64

In [ ]:
# bahasa.value_counts().to_csv('bahasa_distinct.csv')

Lakukan cleaning secara manual dengan google sheet.

In [ ]:
b = pd.read_csv("/content/drive/My Drive/Data Loker/bahasa_mapper.csv")

temp = list()
for idx, row in b.iterrows():
    split = [data.strip() for data in row['target'].split(',')]
    temp += split

pd.Series(temp).value_counts()

inggris           135
indonesia         126
mandarin           58
jepang              9
korea               5
hokien              2
arab                1
tidak tersedia      1
malay               1
jerman              1
dtype: int64

In [ ]:
b.head()

,asli,target
0,bahasa indonesia,indonesia
1,bahasa inggris,inggris
2,tidak tersedia,tidak tersedia
3,bahasa indonesia & english,"indonesia, inggris"
4,bahasa indonesia - english,"indonesia, inggris"


In [ ]:
# create dict mapper
bahasa_mapper = dict()

for idx, row in b.iterrows():
    bahasa_mapper[row['asli']] = row['target']

In [ ]:
# iterate
result = list()
for i in range(len(bahasa)):
    split = [data.strip() for data in bahasa[i].split(',')]
    mapping = pd.Series(split).map(bahasa_mapper).dropna().drop_duplicates()

    result.append(list(mapping))

In [ ]:
# join list
result_bahasa = list()
for res in result:
    r = ', '.join(res).strip()
    result_bahasa.append(r)

In [ ]:
# cek nilai kosong dan isi dengan 'tidak dicantumkan'
for idx, res in enumerate(result_bahasa):
    if res == 'tidak tersedia':
        result_bahasa[idx] = 'tidak dicantumkan'

    if len(res) <= 1:
        result_bahasa[idx] = 'tidak dicantumkan'

In [ ]:
# check null values
data['work_environment_bahasa'] = result_bahasa
data['work_environment_bahasa'].isnull().sum()

0

In [ ]:
data.head()

,platform,job_position,company_name,years_of_experience,company_location,job_description,average_processing_time,company_industry,company_size,work_environment_waktu_bekerja,work_environment_gaya_berpakaian,work_environment_tunjangan,work_environment_bahasa,years_of_experience_cat,work_environment_gaya_berpakaian_contoh
0,jobstreet,SPV FINANCE,PT Axindo Infotama,3,Indonesia - Jakarta Raya - Jakarta Pusat - Ruk...,[SEP][SEP]Usia Max. 35 tahun[SEP]Pendidikan Mi...,6,Retail/Merchandise,51 - 200 pekerja,sabtu,formal,"kesehatan, umum",indonesia,0 - 4 tahun,kemeja polos & dasi
1,jobstreet,Qualitative Research Executive,Perusahaan Dirahasiakan,3,Indonesia - Jakarta Raya,RESPONSIBILITIES:[SEP]Brief Description of Dut...,tidak tersedia,tidak tersedia,tidak spesifik,tidak tersedia,tidak spesifik,tidak tersedia,tidak dicantumkan,0 - 4 tahun,tidak spesifik
2,jobstreet,Field Collection (JS - FC) Wonogiri,OTO Group,0,Indonesia - Jawa Tengah - Wonogiri Wonogoro - ...,[SEP]Lingkup pekerjaan: [SEP]Menjaga account d...,14,Perbankan/Pelayanan Keuangan,lebih dari 5000 pekerja,senin - jumat,bisnis,"kesehatan, bonus",indonesia,0 - 4 tahun,kemeja
3,jobstreet,Perawat di Negara Jerman,PT Jerman Manajemen,1,Indonesia - Jakarta Raya,[SEP]Candidate must possess at least Diploma i...,11,Pendidikan,51 - 200 pekerja,sabtu - kamis,formal,kesehatan,indonesia,0 - 4 tahun,kemeja polos & dasi
4,jobstreet,Field Collection (JS - FC) Pati,OTO Group,0,Indonesia - Jawa Tengah - Pati,[SEP]Lingkup pekerjaan: [SEP]Menjaga account d...,14,Perbankan/Pelayanan Keuangan,lebih dari 5000 pekerja,senin - jumat,bisnis,"kesehatan, bonus",indonesia,0 - 4 tahun,kemeja


## Ekstrak company_location
Menghasilkan atribut baru yaitu <code>pulau</code> dan <code>provinsi</code>

In [ ]:
# data bantuan 
kota_kab = pd.read_csv("/content/drive/My Drive/Data Loker/daftar_kota_kabupaten_id.csv")
kota_kab.head()

,Pulau,Provinsi,Kabupaten/Kota
0,Sumatera,Aceh,Aceh Barat
1,Sumatera,Aceh,Aceh Barat Daya
2,Sumatera,Aceh,Aceh Besar
3,Sumatera,Aceh,Aceh Jaya
4,Sumatera,Aceh,Aceh Selatan


In [ ]:
pulau = kota_kab.Pulau.unique()
provinsi = kota_kab.Provinsi.unique()
pulau, provinsi

(array(['Sumatera', 'Jawa', 'Nusa Tenggara', 'Kalimantan', 'Sulawesi',
        'Maluku', 'Papua'], dtype=object),
 array(['Aceh', 'Sumatera Utara', 'Sumatera Barat', 'Riau',
        'Kepulauan Riau', 'Jambi', 'Bengkulu', 'Sumatera Selatan',
        'Kepulauan Bangka Belitung', 'Lampung', 'Banten', 'Jawa Barat',
        'DKI Jakarta', 'Jawa Tengah', 'DI Yogyakarta', 'Jawa Timur',
        'Bali', 'Nusa Tenggara Barat', 'Nusa Tenggara Timur',
        'Kalimantan Barat', 'Kalimantan Selatan', 'Kalimantan Tengah',
        'Kalimantan Timur', 'Kalimantan Utara', 'Gorontalo',
        'Sulawesi Selatan', 'Sulawesi Barat', 'Sulawesi Tenggara',
        'Sulawesi Tengah', 'Sulawesi Utara', 'Maluku', 'Maluku Utara',
        'Papua', 'Papua Barat'], dtype=object))

In [ ]:
pulau_provinsi = kota_kab[['Pulau', 'Provinsi']].drop_duplicates(subset=['Provinsi']).reset_index(drop=True)
pulau_provinsi['Pulau'] = pulau_provinsi['Pulau'].str.lower()
pulau_provinsi['Provinsi'] = pulau_provinsi['Provinsi'].str.lower()
pulau_provinsi

,Pulau,Provinsi
0,sumatera,aceh
1,sumatera,sumatera utara
2,sumatera,sumatera barat
3,sumatera,riau
4,sumatera,kepulauan riau
5,sumatera,jambi
6,sumatera,bengkulu
7,sumatera,sumatera selatan
8,sumatera,kepulauan bangka belitung
9,sumatera,lampung


In [ ]:
# lihat data company_location, fillna dengan 'tidak diketahui'
location = data['company_location'].str.lower().fillna('tidak diketahui')

location.value_counts()

indonesia - jakarta raya                                                                 1778
tidak diketahui                                                                          1411
indonesia - jakarta raya - jakarta selatan                                                685
indonesia - jawa timur - surabaya                                                         418
indonesia - jakarta raya - jakarta barat                                                  390
                                                                                         ... 
indonesia - jawa timur - pasuruan - prigen                                                  1
indonesia - jawa tengah - semarang, tegal, kudus                                            1
indonesia - jawa timur - surabaya - menganti jeruk                                          1
indonesia - jawa barat - jln. raya sarengseng pabuaran km.3 subang                          1
indonesia - jawa barat - cikarang - pt prospect motor jl mh 

- <code>provinsi</code> didapatkan pada data kedua
- <code>pulau</code> didapatkan dengan mencocokkan pada data bantuan

In [ ]:
# ambil provinsi, kemudian cocokkan dengan data bantuan untuk dapatkan pulau

pulau, provinsi = list(), list()

for idx, loc in enumerate(location):
    split_text = loc.split('-')

    prov = split_text[1:2][0].strip() if len(split_text) > 1 else split_text[0]

    # replace 'jakarta raya' -> 'dki jakarta'
    # replace 'yogyakarta' -> 'di yogyakarta'
    # replace 'bangka belitung' -> 'kepulauan bangka belitung'
    prov = 'dki jakarta' if prov == 'jakarta raya' else prov
    prov = 'di yogyakarta' if prov == 'yogyakarta' else prov
    prov = 'kepulauan bangka belitung' if prov == 'bangka belitung' else prov

    if prov in pulau_provinsi['Provinsi'].values:
        idx = pulau_provinsi.index[pulau_provinsi['Provinsi'] == prov][0]
        pul = pulau_provinsi.iloc[idx]['Pulau']
    else:
        pul = 'tidak diketahui'

    provinsi.append(prov)
    pulau.append(pul)

In [ ]:
pd.Series(provinsi).value_counts()

dki jakarta                  5985
jawa barat                   1771
tidak diketahui              1411
jawa timur                   1296
banten                       1219
jawa tengah                   548
sumatera utara                225
di yogyakarta                 216
bali                          214
kalimantan timur              159
kepulauan riau                126
sulawesi selatan              121
sumatera selatan              120
riau                           86
kalimantan selatan             72
kalimantan barat               54
kalimantan tengah              45
lampung                        43
sulawesi utara                 42
sulawesi tengah                40
nusa tenggara barat            37
jambi                          37
kalimantan utara               22
aceh                           21
sulawesi tenggara              20
nusa tenggara timur            18
sumatera barat                 17
papua                          16
maluku utara                   14
papua barat   

In [ ]:
pd.Series(pulau).value_counts()

jawa               11035
tidak diketahui     1434
sumatera             690
kalimantan           352
nusa tenggara        269
sulawesi             231
papua                 28
maluku                20
dtype: int64

In [ ]:
# insert into dataframe
data['provinsi'] = provinsi
data['pulau'] = pulau

# drop company_location
data = data.drop(['company_location'], axis=1)

In [ ]:
data.head()

,platform,job_position,company_name,years_of_experience,job_description,average_processing_time,company_industry,company_size,work_environment_waktu_bekerja,work_environment_gaya_berpakaian,work_environment_tunjangan,work_environment_bahasa,years_of_experience_cat,work_environment_gaya_berpakaian_contoh,provinsi,pulau
0,jobstreet,SPV FINANCE,PT Axindo Infotama,3,[SEP][SEP]Usia Max. 35 tahun[SEP]Pendidikan Mi...,6,Retail/Merchandise,51 - 200 pekerja,sabtu,formal,"kesehatan, umum",indonesia,0 - 4 tahun,kemeja polos & dasi,dki jakarta,jawa
1,jobstreet,Qualitative Research Executive,Perusahaan Dirahasiakan,3,RESPONSIBILITIES:[SEP]Brief Description of Dut...,tidak tersedia,tidak tersedia,tidak spesifik,tidak tersedia,tidak spesifik,tidak tersedia,tidak dicantumkan,0 - 4 tahun,tidak spesifik,dki jakarta,jawa
2,jobstreet,Field Collection (JS - FC) Wonogiri,OTO Group,0,[SEP]Lingkup pekerjaan: [SEP]Menjaga account d...,14,Perbankan/Pelayanan Keuangan,lebih dari 5000 pekerja,senin - jumat,bisnis,"kesehatan, bonus",indonesia,0 - 4 tahun,kemeja,jawa tengah,jawa
3,jobstreet,Perawat di Negara Jerman,PT Jerman Manajemen,1,[SEP]Candidate must possess at least Diploma i...,11,Pendidikan,51 - 200 pekerja,sabtu - kamis,formal,kesehatan,indonesia,0 - 4 tahun,kemeja polos & dasi,dki jakarta,jawa
4,jobstreet,Field Collection (JS - FC) Pati,OTO Group,0,[SEP]Lingkup pekerjaan: [SEP]Menjaga account d...,14,Perbankan/Pelayanan Keuangan,lebih dari 5000 pekerja,senin - jumat,bisnis,"kesehatan, bonus",indonesia,0 - 4 tahun,kemeja,jawa tengah,jawa


Lakukan drop pada data yang berlokasi tidak di Indonesia:
- seluruh singapore
- national capital reg
- selangor
- papua new guinea
- central
- uganda
- taipei
- east
- north

In [ ]:
deleted = ['seluruh singapore',
           'national capital reg',
           'selangor',
           'papua new guinea',
           'central',
           'uganda',
           'taipei',
           'east',
           'north']

# select data yang tanpa nama provinsi di atas
data = data[~data['provinsi'].isin(deleted)]

data['provinsi'].value_counts()

dki jakarta                  5985
jawa barat                   1771
tidak diketahui              1411
jawa timur                   1296
banten                       1219
jawa tengah                   548
sumatera utara                225
di yogyakarta                 216
bali                          214
kalimantan timur              159
kepulauan riau                126
sulawesi selatan              121
sumatera selatan              120
riau                           86
kalimantan selatan             72
kalimantan barat               54
kalimantan tengah              45
lampung                        43
sulawesi utara                 42
sulawesi tengah                40
jambi                          37
nusa tenggara barat            37
kalimantan utara               22
aceh                           21
sulawesi tenggara              20
nusa tenggara timur            18
sumatera barat                 17
papua                          16
maluku utara                   14
papua barat   

In [ ]:
data.shape

(14036, 16)

In [ ]:
data.isnull().sum()

platform                                   0
job_position                               0
company_name                               0
years_of_experience                        0
job_description                            0
average_processing_time                    0
company_industry                           0
company_size                               0
work_environment_waktu_bekerja             0
work_environment_gaya_berpakaian           0
work_environment_tunjangan                 0
work_environment_bahasa                    0
years_of_experience_cat                    0
work_environment_gaya_berpakaian_contoh    0
provinsi                                   0
pulau                                      0
dtype: int64

In [ ]:
data.head()

,platform,job_position,company_name,years_of_experience,job_description,average_processing_time,company_industry,company_size,work_environment_waktu_bekerja,work_environment_gaya_berpakaian,work_environment_tunjangan,work_environment_bahasa,years_of_experience_cat,work_environment_gaya_berpakaian_contoh,provinsi,pulau
0,jobstreet,SPV FINANCE,PT Axindo Infotama,3,[SEP][SEP]Usia Max. 35 tahun[SEP]Pendidikan Mi...,6,Retail/Merchandise,51 - 200 pekerja,sabtu,formal,"kesehatan, umum",indonesia,0 - 4 tahun,kemeja polos & dasi,dki jakarta,jawa
1,jobstreet,Qualitative Research Executive,Perusahaan Dirahasiakan,3,RESPONSIBILITIES:[SEP]Brief Description of Dut...,tidak tersedia,tidak tersedia,tidak spesifik,tidak tersedia,tidak spesifik,tidak tersedia,tidak dicantumkan,0 - 4 tahun,tidak spesifik,dki jakarta,jawa
2,jobstreet,Field Collection (JS - FC) Wonogiri,OTO Group,0,[SEP]Lingkup pekerjaan: [SEP]Menjaga account d...,14,Perbankan/Pelayanan Keuangan,lebih dari 5000 pekerja,senin - jumat,bisnis,"kesehatan, bonus",indonesia,0 - 4 tahun,kemeja,jawa tengah,jawa
3,jobstreet,Perawat di Negara Jerman,PT Jerman Manajemen,1,[SEP]Candidate must possess at least Diploma i...,11,Pendidikan,51 - 200 pekerja,sabtu - kamis,formal,kesehatan,indonesia,0 - 4 tahun,kemeja polos & dasi,dki jakarta,jawa
4,jobstreet,Field Collection (JS - FC) Pati,OTO Group,0,[SEP]Lingkup pekerjaan: [SEP]Menjaga account d...,14,Perbankan/Pelayanan Keuangan,lebih dari 5000 pekerja,senin - jumat,bisnis,"kesehatan, bonus",indonesia,0 - 4 tahun,kemeja,jawa tengah,jawa


## Ekstrak job_position
Dilakukan pengelompokkan menjadi job type, supaya distinct.

Menuju colab [2_2_Ekstrak_job_position](https://colab.research.google.com/drive/1Mu-AeZdReKU5u7-_xyX7gaOXVqOMYZ5M?usp=sharing).

> **Question:** necessary?

In [ ]:
# convert to lower case dan fillna dengan string 'tidak tersedia'
job_position = jobstreet['job_position'].str.lower().fillna('tidak tersedia')

job_position.value_counts()[:15]

sales executive          80
accounting staff         68
digital marketing        56
marketing                49
sales manager            48
accounting               44
project manager          43
graphic designer         42
sales                    42
staff accounting         38
marketing manager        37
marketing executive      36
sales engineer           34
accounting supervisor    32
web developer            32
Name: job_position, dtype: int64

In [ ]:
# pd.Series(temp).value_counts().to_csv('job_position_distinct.csv')

## Ekstrak job_description

Hasil ekstrak adalah:
1. **Skill**: keahlian yang dibutuhkan
2. **Pendidikan**: kualifikasi pendidikan

Proses dilakukan pada
- [Colab 2_3_1 Ekstrak job_description (pendidikan)](https://colab.research.google.com/drive/1Ddf22nhb-4CYBhlyO2Gl6xRkBAG4Fsst?usp=sharing)
- [Colab 2_3_2 Ekstrak job_description (skill - english)](https://colab.research.google.com/drive/13OT_WSJYNh0D0vkBPfFWEME8Za5Ojblh?usp=sharing)
- [Colab 2_3_3 Ekstrak job_description (skill - indonesia)](https://colab.research.google.com/drive/1UFUKEMnLU1JgWvSnYcvYNG71FiVeEd4-?usp=sharing)
- [Colab 2_3_4 Ekstrak job_description (skill - en&id)](https://colab.research.google.com/drive/1KcdFkTxPW8c03txSBIXpS_fZEF73Ws77?usp=sharing)

In [ ]:
# convert to lower case dan fillna dengan string 'kosong'
job_description = jobstreet['job_description'].str.lower().fillna('kosong')

job_description.head()

0    [sep][sep]usia max. 35 tahun[sep]pendidikan mi...
1    responsibilities:[sep]brief description of dut...
2    [sep]lingkup pekerjaan: [sep]menjaga account d...
3    [sep]candidate must possess at least diploma i...
4    [sep]lingkup pekerjaan: [sep]menjaga account d...
Name: job_description, dtype: object

In [ ]:
job_description[0]

'[sep][sep]usia max. 35 tahun[sep]pendidikan min. d3[sep]pengalaman min. 3 tahun sebagai senior staff atau spv finance dari perusahaan distributor atau sejenisnya[sep]mempunyai leadership yang kuat, bertanggung jawab & jujur[sep]memiliki pengelolaan dan analisa yg baik, teliti dan cekatan[sep]bersedia visit ke kantor cabang secara rutin[sep]penempatan head office cempaka mas jakarta pusat[sep][sep]tugas & tanggung jawab :[sep][sep]melakukan supervisi tim admin finance(a/r, retur, refund & a/p)[sep]melakukan kontrol aging a/r & limit a/r[sep]membuat schedule a/p payment[sep]melakukan kontrol terhadap retur/refund(cn/dn)[sep]melakukan follow up terkait program-program & point reward[sep]melakukan kordinasi dengan pihak terkait[sep]memaintance utang piutang terhadap dealer-dealer[sep][sep]'

In [ ]:
z = jobstreet[jobstreet['job_position'] == 'Web Developer']
f = z[['job_position', 'job_description']]
f['job_description'][4013]

'[SEP] Usia Maksimal 40 Tahun [SEP]Pengalaman minimal 1 Tahun [SEP]Menguasai /Familiar dengan PHP,,MYSQL,HTML,CSS,COREL,PHOTOSHOP,WEB PROGRAMMING, dan pengoperasian dengan IMAC [SEP]Diutamakan ASP, Java, PHP, Android,IOS[SEP]Komunikatif, Kreatif, dan inisiatif[SEP]Good Personal, Good looking [SEP]Bersedia membawa Company Profile personal saat ada proses seleksi [SEP]Penempatan di Tangerang [SEP][SEP]Tugas dan Tanggung Jawab [SEP][SEP]Membuat dan mengembangkan website[SEP]Melakukan Maintanance periodik terhadap website[SEP]Membuat konten website yang fungsional sehingga dapat menyiapkan informasi tentang produk/ program promosi , aktivitas Perusahaan dengan customer , Layanan kepada Customer [SEP][SEP] [SEP]'

### pendidikan

In [ ]:
# cari job description yang mengandung 
# kata kunci 'pendidikan', 'degree', 'lulusan'
temp = list(job_description.values)

count = 0
for idx, job_desc in enumerate(temp):
    if 'pendidikan' in job_desc:
        count += 1
    elif 'degree' in job_desc:
        count += 1
    elif 'lulusan' in job_desc:
        count += 1
count

9698

### skill

In [ ]:
pass

# Export csv

In [ ]:
data.head()

,platform,job_position,company_name,years_of_experience,job_description,average_processing_time,company_industry,company_size,work_environment_waktu_bekerja,work_environment_gaya_berpakaian,work_environment_tunjangan,work_environment_bahasa,years_of_experience_cat,work_environment_gaya_berpakaian_contoh,provinsi,pulau
0,jobstreet,SPV FINANCE,PT Axindo Infotama,3,[SEP][SEP]Usia Max. 35 tahun[SEP]Pendidikan Mi...,6,Retail/Merchandise,51 - 200 pekerja,sabtu,formal,"kesehatan, umum",indonesia,0 - 4 tahun,kemeja polos & dasi,dki jakarta,jawa
1,jobstreet,Qualitative Research Executive,Perusahaan Dirahasiakan,3,RESPONSIBILITIES:[SEP]Brief Description of Dut...,tidak tersedia,tidak tersedia,tidak spesifik,tidak tersedia,tidak spesifik,tidak tersedia,tidak dicantumkan,0 - 4 tahun,tidak spesifik,dki jakarta,jawa
2,jobstreet,Field Collection (JS - FC) Wonogiri,OTO Group,0,[SEP]Lingkup pekerjaan: [SEP]Menjaga account d...,14,Perbankan/Pelayanan Keuangan,lebih dari 5000 pekerja,senin - jumat,bisnis,"kesehatan, bonus",indonesia,0 - 4 tahun,kemeja,jawa tengah,jawa
3,jobstreet,Perawat di Negara Jerman,PT Jerman Manajemen,1,[SEP]Candidate must possess at least Diploma i...,11,Pendidikan,51 - 200 pekerja,sabtu - kamis,formal,kesehatan,indonesia,0 - 4 tahun,kemeja polos & dasi,dki jakarta,jawa
4,jobstreet,Field Collection (JS - FC) Pati,OTO Group,0,[SEP]Lingkup pekerjaan: [SEP]Menjaga account d...,14,Perbankan/Pelayanan Keuangan,lebih dari 5000 pekerja,senin - jumat,bisnis,"kesehatan, bonus",indonesia,0 - 4 tahun,kemeja,jawa tengah,jawa


In [ ]:
# create column id_loker
data['id_loker'] = [i+1 for i in range(len(data))]

In [ ]:
data.columns

Index(['platform', 'job_position', 'company_name', 'years_of_experience',
       'job_description', 'average_processing_time', 'company_industry',
       'company_size', 'work_environment_waktu_bekerja',
       'work_environment_gaya_berpakaian', 'work_environment_tunjangan',
       'work_environment_bahasa', 'years_of_experience_cat',
       'work_environment_gaya_berpakaian_contoh', 'provinsi', 'pulau',
       'id_loker'],
      dtype='object')

In [ ]:
# reorder column
cols = ['id_loker',
        'pulau',
        'provinsi',
        'job_position',
        'company_name',
        'years_of_experience',
        'years_of_experience_cat',
        'job_description',
        'average_processing_time',
        'company_industry',
        'company_size',
        'work_environment_waktu_bekerja',
        'work_environment_gaya_berpakaian',
        'work_environment_gaya_berpakaian_contoh',
        'work_environment_tunjangan',
        'work_environment_bahasa',
        'platform']

data = data[cols]
data.head()

,id_loker,pulau,provinsi,job_position,company_name,years_of_experience,years_of_experience_cat,job_description,average_processing_time,company_industry,company_size,work_environment_waktu_bekerja,work_environment_gaya_berpakaian,work_environment_gaya_berpakaian_contoh,work_environment_tunjangan,work_environment_bahasa,platform
0,1,jawa,dki jakarta,SPV FINANCE,PT Axindo Infotama,3,0 - 4 tahun,[SEP][SEP]Usia Max. 35 tahun[SEP]Pendidikan Mi...,6,Retail/Merchandise,51 - 200 pekerja,sabtu,formal,kemeja polos & dasi,"kesehatan, umum",indonesia,jobstreet
1,2,jawa,dki jakarta,Qualitative Research Executive,Perusahaan Dirahasiakan,3,0 - 4 tahun,RESPONSIBILITIES:[SEP]Brief Description of Dut...,tidak tersedia,tidak tersedia,tidak spesifik,tidak tersedia,tidak spesifik,tidak spesifik,tidak tersedia,tidak dicantumkan,jobstreet
2,3,jawa,jawa tengah,Field Collection (JS - FC) Wonogiri,OTO Group,0,0 - 4 tahun,[SEP]Lingkup pekerjaan: [SEP]Menjaga account d...,14,Perbankan/Pelayanan Keuangan,lebih dari 5000 pekerja,senin - jumat,bisnis,kemeja,"kesehatan, bonus",indonesia,jobstreet
3,4,jawa,dki jakarta,Perawat di Negara Jerman,PT Jerman Manajemen,1,0 - 4 tahun,[SEP]Candidate must possess at least Diploma i...,11,Pendidikan,51 - 200 pekerja,sabtu - kamis,formal,kemeja polos & dasi,kesehatan,indonesia,jobstreet
4,5,jawa,jawa tengah,Field Collection (JS - FC) Pati,OTO Group,0,0 - 4 tahun,[SEP]Lingkup pekerjaan: [SEP]Menjaga account d...,14,Perbankan/Pelayanan Keuangan,lebih dari 5000 pekerja,senin - jumat,bisnis,kemeja,"kesehatan, bonus",indonesia,jobstreet


In [ ]:
data.to_csv('jobstreet_clean_tahap1.csv', index=False)

In [ ]:
data.isnull().sum()

id_loker                                   0
pulau                                      0
provinsi                                   0
job_position                               0
company_name                               0
years_of_experience                        0
years_of_experience_cat                    0
job_description                            0
average_processing_time                    0
company_industry                           0
company_size                               0
work_environment_waktu_bekerja             0
work_environment_gaya_berpakaian           0
work_environment_gaya_berpakaian_contoh    0
work_environment_tunjangan                 0
work_environment_bahasa                    0
platform                                   0
dtype: int64